In [6]:
from collections import defaultdict
from itertools import product

class CKSAAPFeatureExtraction:
    def __init__(self, kmax=5):
        self.amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
        self.kmax = kmax

    def calculate_cksaap_features(self, sequence):
        """
        Calculate the CKSAAP features for a given protein sequence.

        :param sequence: The protein sequence as a string.
        :return: A list of CKSAAP feature values.
        """
        # Generate all possible amino acid pairs
        amino_acid_pairs = [''.join(pair) for pair in product(self.amino_acids, repeat=2)]

        # Initialize a dictionary to store the counts of each k-spaced amino acid pair
        cksaap_counts = {pair: [0] * (self.kmax + 1) for pair in amino_acid_pairs}

        # Calculate the counts of each k-spaced amino acid pair
        for k in range(self.kmax + 1):
            for i in range(len(sequence) - k - 1):
                pair = sequence[i] + sequence[i + k + 1]
                if pair in cksaap_counts:
                    cksaap_counts[pair][k] += 1

        # Calculate the window size for each k
        window_sizes = [len(sequence) - k - 1 for k in range(self.kmax + 1)]

        # Calculate the CKSAAP feature values using the formula
        cksaap_features = []
        for pair in amino_acid_pairs:
            for k in range(self.kmax + 1):
                count = cksaap_counts[pair][k]
                window_size = window_sizes[k]
                if window_size > 0:
                    cksaap_features.append(count / window_size)
                else:
                    cksaap_features.append(0)

        return cksaap_features

# # Example usage
# sequence = "ACDEFGHIKLMNPQRSTVWY"
# extractor = CKSAAPFeatureExtraction(kmax=5)
# cksaap_features = extractor.calculate_cksaap_features(sequence)
# print(cksaap_features)

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, matthews_corrcoef
from sklearn.model_selection import LeaveOneOut
import numpy as np

# Load the dataset
data = pd.read_excel('../data/Final_2Sm_modified_with_sequences.xlsx')

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder to the folding_type column and transform it to numeric labels
data['folding_type'] = label_encoder.fit_transform(data['folding_type'])

# Now, when you extract labels for model training:
labels = data['folding_type'].values

# Initialize the FeatureExtraction class
feature_extraction = CKSAAPFeatureExtraction()

# Feature extraction using AAC with length
features = np.array([feature_extraction.calculate_cksaap_features(seq) for seq in data['sequence']])

# Support Vector Machine (SVM) Classifier

In [3]:
# SVM with Leave-One-Out Cross-Validation (LOOCV)
loo = LeaveOneOut()
y_true, y_pred = [], []
for train_index, test_index in loo.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)
    y_pred.append(clf.predict(X_test)[0])
    y_true.append(y_test[0])

In [4]:
# Calculate and display the confusion matrix
from ClassificationMatrix import ClassificationMatrix

cm = ClassificationMatrix(y_true, y_pred, 'KAAC')
cm.evaluate()

Confusion Matrix: $KAAC
[[89  0]
 [52  0]]

Accuracy (ACC): 0.63
Matthews Correlation Coefficient (MCC): 0.00

Classification Report:
              precision    recall  f1-score   support

           0       0.63      1.00      0.77        89
           1       0.00      0.00      0.00        52

    accuracy                           0.63       141
   macro avg       0.32      0.50      0.39       141
weighted avg       0.40      0.63      0.49       141


# Random Forest Classifier implementation with Hyperparameter Tuning

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Grid search with cross validation setup
grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid, cv=3, scoring='accuracy')

# Fit the grid search to find the best parameters
grid_search.fit(features, labels)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters: ", best_params)
print("Best score: ", best_score)

# Using the best parameters with LOOCV
best_rf = RandomForestClassifier(**best_params, random_state=42)
loo = LeaveOneOut()
y_true, y_pred = [], []

for train_index, test_index in loo.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    best_rf.fit(X_train, y_train)
    y_pred.append(best_rf.predict(X_test)[0])
    y_true.append(y_test[0])

Best parameters:  {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
Best score:  0.7304964539007092


In [9]:
# Calculate and display the confusion matrix
from ClassificationMatrix import ClassificationMatrix

cm = ClassificationMatrix(y_true, y_pred, 'CKSAAP')
cm.evaluate()

Confusion Matrix: $CKSAAP
[[84  5]
 [33 19]]

Accuracy (ACC): 0.73
Matthews Correlation Coefficient (MCC): 0.40

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.94      0.82        89
           1       0.79      0.37      0.50        52

    accuracy                           0.73       141
   macro avg       0.75      0.65      0.66       141
weighted avg       0.75      0.73      0.70       141
